In [7]:
import json
from collections import defaultdict
from mongo_handler import MongoHandler

def get_percentage_changes_training_corpus(k_fold=1, threshold_perc_change = 300):
    mongo_handler_obj = MongoHandler()
    mongo_handler_obj.connect_to_mongo()
    db = mongo_handler_obj.get_database()
    list_of_input_data = []
    data = db["input_data"].find({"is_used": True, "year": {"$ne": 2021}})
    list_of_ids_to_remove = []
    for item in data:
        list_of_input_data.append(item)
            
    curr_min_standard = 10000
    curr_max_standard = -10000
    curr_min_minmax = 10000
    curr_max_minmax = -10000
    curr_min = 10000000
    curr_max = -10000000
    dict_of_percentage_changes = defaultdict(list)
    for input_data in list_of_input_data:
        
        if input_data["percentage_change"] > threshold_perc_change:
            list_of_ids_to_remove.append(str(input_data["_id"]))
        if input_data["percentage_change_scaled_standard"][str(k_fold)] < curr_min_standard:
            curr_min_standard = input_data["percentage_change_scaled_standard"][str(k_fold)]
        if input_data["percentage_change_scaled_standard"][str(k_fold)] > curr_max_standard:
            curr_max_standard = input_data["percentage_change_scaled_standard"][str(k_fold)]
        if input_data["percentage_change_scaled_min_max"][str(k_fold)] < curr_min_minmax:
            curr_min_minmax = input_data["percentage_change_scaled_min_max"][str(k_fold)]
        if input_data["percentage_change_scaled_min_max"][str(k_fold)] > curr_max_minmax:
            curr_max_minmax = input_data["percentage_change_scaled_min_max"][str(k_fold)]
        if input_data["percentage_change"] < curr_min:
            curr_min= input_data["percentage_change"]
        if input_data["percentage_change"] > curr_max:
            curr_max= input_data["percentage_change"]
            
        dict_of_percentage_changes["percentage_change"].append(input_data["percentage_change"])
        dict_of_percentage_changes["percentage_change_min_max"].append(input_data["percentage_change_scaled_min_max"][str(k_fold)])
        dict_of_percentage_changes["percentage_change_standard"].append(input_data["percentage_change_scaled_standard"][str(k_fold)])
            
    return dict_of_percentage_changes, list_of_ids_to_remove
    

In [8]:
dict_of_percentage_changes_train, list_of_ids_to_remove = get_percentage_changes_training_corpus()

In [9]:
import plotly.figure_factory as ff
import numpy as np

hist_data = [dict_of_percentage_changes_train["percentage_change"], dict_of_percentage_changes_train["percentage_change_standard"], dict_of_percentage_changes_train["percentage_change_min_max"]]

group_labels = ['percentage_change', "percentage_change_standard", "percentage_change_min_max"]

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [10]:
import plotly.figure_factory as ff
import numpy as np

hist_data = [dict_of_percentage_changes_val["percentage_change"], dict_of_percentage_changes_val["percentage_change_standard"], dict_of_percentage_changes_val["percentage_change_min_max"]]

group_labels = ['percentage_change', 'percentage_change_standard', 'percentage_change_min_max']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels)
fig.show()

NameError: name 'dict_of_percentage_changes_val' is not defined

## Pre-training Adapter results

In [6]:
import pandas as pd

val_df = pd.read_csv("data/pre_training_adapter/run-runs_sec_adapter_maxlen-64_batch-64_lr-5e-05_warmup-0_epoch-100_-tag-eval_loss (1).csv")
train_df = pd.read_csv("data/pre_training_adapter/run-runs_sec_adapter_maxlen-64_batch-64_lr-5e-05_warmup-0_epoch-100_-tag-loss.csv")

df_merged = pd.merge(train_df, val_df, how="inner", on="Step", suffixes=(None, "_val"))

In [8]:
import plotly.express as px

fig = px.line(df_merged, x="Step", y=["Value", "Value_val"], title=f"Pre-training Adapter train vs val loss")
fig.show()

In [29]:
from math import pow

"""
Using minmax
"""

# Possible predictions
pred_low = 0.21
pred = 0.39

# Possible real
real = 0.3

print(pow((pred_low-real), 2))
print(pow((pred-real), 2))

0.0081
0.008100000000000005


## Try KPI model

In [24]:
from torch import nn

class KPIModel(nn.Module):
    def __init__(self):
        super(KPIModel, self).__init__()
        self.input_size = 116
        self.num_classes = 1
        self.hidden_size = 64
        self.hidden_layers = 1

        if self.hidden_layers == 0:
            self.layers = nn.Linear(self.input_size, self.num_classes)
        elif self.hidden_layers == 1:
            self.layers = nn.Sequential(
                nn.Linear(self.input_size, self.hidden_size),
                nn.Dropout(0.2),
                nn.ReLU(),
                nn.Linear(self.hidden_size, self.num_classes),
            )

    def forward(self, x, labels=None):
        outputs = self.layers(x)
        
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(outputs, labels)
            outputs = (loss, outputs)
            
        return outputs

In [25]:
import torch
model = KPIModel()
model.load_state_dict(torch.load("D:/PythonProjects/K-Adapter/output/kpi_symbolic_kpi-symbolic_percentage_change_standard_kfold-1_batch-64_lr-5e-05_warmup-0_epoch-2000.0_comment-/kpi_pytorch_model.bin", map_location=torch.device('cpu')))
model.eval()

KPIModel(
  (layers): Sequential(
    (0): Linear(in_features=116, out_features=64, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [34]:
import pickle
from collections import defaultdict
from mongo_handler import MongoHandler

list_of_features_dicts = [
            "fundamental_data_imputed_full",
            "fundamental_data_diff_self_t_1",
            "fundamental_data_diff_self_t_2",
            "fundamental_data_diff_industry_t",
            "fundamental_data_diff_industry_t_1",
            "fundamental_data_diff_industry_t_2",
        ]

def run_kpi_model_val_data_per_k_fold(model, k_fold=1, perc_change_type="standard"):
    model.eval()
    mongo_handler_obj = MongoHandler()
    mongo_handler_obj.connect_to_mongo()
    db = mongo_handler_obj.get_database()
    data = db["input_data"].find({"is_used": True, f"k_fold_config.{k_fold}": "val"})
    scaler = db["storage"].find_one({"name": perc_change_type, "k_fold": k_fold})
    # for min_max_storage in storages_min_max:
    scaler = pickle.loads(scaler["dumped_object"])

    dict_of_results = defaultdict(dict)
    for input_data in data:
        list_features = []
        for features_dict in list_of_features_dicts:
            list_features += list(input_data[features_dict].values())
        if input_data["is_filing_on_time"]:
            list_features += [0, 1]
        else:
            list_features += [1, 0]
        
        with torch.no_grad():
            curr_loss, curr_output = model(torch.FloatTensor(list_features), torch.FloatTensor([input_data[f"percentage_change_scaled_{perc_change_type}"][str(k_fold)]]))
        
        inversed_output = scaler.inverse_transform(curr_output.reshape(1,-1))
        
        dict_of_results[str(input_data["_id"])]['loss'] = curr_loss
        dict_of_results[str(input_data["_id"])]['actual'] = input_data["percentage_change"]
        dict_of_results[str(input_data["_id"])]['predicted_inv'] = inversed_output
        dict_of_results[str(input_data["_id"])]['act_pred_diff_inv'] = abs(inversed_output - input_data["percentage_change"])
        dict_of_results[str(input_data["_id"])]['actual_scaled'] = input_data[f"percentage_change_scaled_{perc_change_type}"][str(k_fold)]
        dict_of_results[str(input_data["_id"])]['predicted_scaled'] = curr_output.item()
        dict_of_results[str(input_data["_id"])]['act_pred_diff_scaled'] = abs(curr_output.item() - input_data[f"percentage_change_scaled_{perc_change_type}"][str(k_fold)])
    
    return dict_of_results

In [35]:
import numpy as np

dict_of_results = run_kpi_model_val_data_per_k_fold(model, 1, "standard")

In [36]:
dict_of_results

defaultdict(dict,
            {'62795155199c6e379f89949c': {'loss': tensor(0.4013),
              'actual': -23.060480262395664,
              'predicted_inv': array([[1.11198025]]),
              'act_pred_diff_inv': array([[24.17246051]]),
              'actual_scaled': -0.7463434015510451,
              'predicted_scaled': -0.11289053410291672,
              'act_pred_diff_scaled': 0.6334528674481283},
             '62795156199c6e379f8994a1': {'loss': tensor(0.2450),
              'actual': -16.504501348086634,
              'predicted_inv': array([[2.3827273]]),
              'act_pred_diff_inv': array([[18.88722865]]),
              'actual_scaled': -0.5745403019046252,
              'predicted_scaled': -0.07958989590406418,
              'act_pred_diff_scaled': 0.49495040600056106},
             '62795156199c6e379f8994a6': {'loss': tensor(5.7527),
              'actual': 98.70129870129867,
              'predicted_inv': array([[7.17603114]]),
              'act_pred_diff_inv': ar

In [29]:
mongo_handler_obj = MongoHandler()
mongo_handler_obj.connect_to_mongo()
db = mongo_handler_obj.get_database()
    
min_max_scaler = db["storage"].find_one({"name": "standard", "k_fold": 1})
    # for min_max_storage in storages_min_max:
min_max_scaler = pickle.loads(min_max_scaler["dumped_object"])

In [33]:
min_max_scaler.inverse_transform([[-0.4139048159122467]])

array([[-10.37467679]])